In [2]:
import google.generativeai as genai

genai.configure(api_key="<your key>")


def get_completion(prompt):
    """
    Get the completion for a given prompt using the specified model.
    Returns the answer with the highest score.
    """
    model = genai.GenerativeModel(
            "models/gemini-1.5-flash",
            system_instruction="You are a user.",
        )
    response = model.generate_content(prompt)
    return response.text

# Effective Code Writing Prompts

## Prompt: "Help me build my data model"

At some point, you'll have to step in and make some decisions. (check also what if I want it as SQLModel)

In [4]:
prompt = f"""
Can you help me build a data model?  Here are the entities that I have so far:

- User
- Restaurant
- Location
- Reservation

I am building an app where users can search for restaurants near them and book reservations.

Show me what this data model should look like and how I can improve it.
"""
print(get_completion(prompt))

This data model uses SQLModel and addresses potential improvements for a user-restaurant reservation system.  It emphasizes relationships and data integrity.

```python
from typing import Optional, List

from sqlmodel import Field, SQLModel, Relationship


class Location(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    address: str
    city: str
    state: str
    zip_code: str


class Restaurant(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    location_id: int = Field(foreign_key="location.id")  # Foreign key to Location
    location: Location = Relationship(back_populates="restaurants") #backref
    description: Optional[str] = None
    phone_number: Optional[str] = None
    # Add other relevant restaurant attributes (e.g., cuisine, price range, hours)


class User(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    username: str = Field(unique=True)
    

## Prompt: "Build me a utility function that does XYZ"

In [5]:
prompt = """
Write a python function to find the Fibonacci series of n using recursion. 
The function should take these inputs: input-variables and return desired-output.
"""
print(get_completion(prompt))

```python
def fibonacci_recursive(n):
    """
    Calculates the Fibonacci sequence up to n terms using recursion.

    Args:
        n: The number of terms to generate in the Fibonacci sequence.  Must be a non-negative integer.

    Returns:
        A list containing the first n Fibonacci numbers.  Returns an empty list if n is 0 or less.
        Raises ValueError if input is not a non-negative integer.

    """
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer.")
    if n <= 1:
        return [0] * n  #Handle base cases 0 and 1 efficiently
    else:
        list_fib = fibonacci_recursive(n-1)
        list_fib.append(list_fib[-1] + list_fib[-2] if len(list_fib) > 1 else 1) #efficiently append next number
        return list_fib

# Get input from the user.  Error handling included.
while True:
    try:
        num_terms = int(input("Enter the number of terms for the Fibonacci sequence: "))
        if num_terms < 0:
            print(

## Prompt: "Build me a unittest"

In [7]:
function = f"""
    def fibonacci(n):
        if n <= 0:
            return []
        elif n == 1:
            return [0]
        elif n == 2:
            return [0, 1]
        else:
            fib_series = fibonacci(n-1)
            fib_series.append(fibonacci(n-1)[-1] + fibonacci(n-1)[-2])
            return fib_series
"""

prompt = f"""
Build me a unit test for the following function: '''{function}'''
Use parameterized to validate several parameters options
add edge cases to the test (very large number for example)"""
# Use pytest over unittest"""

print(get_completion(prompt))

```python
import pytest
from parameterized import parameterized

def fibonacci(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    else:
        fib_series = fibonacci(n-1)
        fib_series.append(fib_series[-1] + fib_series[-2])
        return fib_series

@parameterized.expand([
    (0, []),
    (1, [0]),
    (2, [0, 1]),
    (3, [0, 1, 1]),
    (5, [0, 1, 1, 2, 3]),
    (10, [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]),
    (20, [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]),
    #Edge case: large number.  May need to adjust depending on system resources and recursion depth limits.
    (30, [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229]),

])
def test_fibonacci_positive(n, expected):
    assert fibonacci(n) == expected


#Test for negative input (edge case)
def test_fibonacci_negat

# Prompts for Debugging and Code Review

## Prompt: "Fix my code"

In [ ]:
def some_function(input):
    for i in range(len(input)):
        if input[i] == 2:
            del input[i]
    return input

list_01 = [1,2,4,5,6]
some_function(list_01)

**Error: Modifying a list while iterating over it**

In [11]:
code = f"""
def some_function(input):
    for i in range(len(input)):
        if input[i] == 2:
            del input[i]
    return input

list_01 = [1,2,4,5,6]
some_function(list_01)"""

prompt = f"""
Find the errors in my code: '''{code}''' and fix it
"""
print(get_completion(prompt))

The primary error in the code lies within the `some_function`.  When you use `del input[i]` inside a loop that's iterating over the indices of `input`, you modify the list's length while simultaneously trying to access indices that no longer exist after the deletion. This leads to skipping elements and potentially IndexError exceptions.


Here's a breakdown of why it's wrong and a few ways to fix it:

**Why it's wrong:**

Imagine the list is `[1, 2, 4, 5, 6]`.

1. The loop starts at `i = 0`. `input[0]` (1) is not 2, so it continues.
2. `i` becomes 1. `input[1]` (2) is 2, so `del input[1]` is executed.  The list becomes `[1, 4, 5, 6]`.
3. `i` increments to 2.  The loop now tries to access `input[2]`, which is *5* (because the list has been modified).  The element that was originally at index 2 (4) is now skipped.


**Correct Approaches:**

There are several ways to correctly remove elements from a list while iterating:

**1. Iterating over a copy:**

This is the simplest and often most 

## Prompt: "Can you improve my code?"

In [13]:
code = f"""
def some_function(input):
    new_list = []
    for item in input_list:
        if item != 2:
            new_list.append(item)
    return new_list

list_01 = [1,2,4,5,6]
some_function(list_01)
"""

prompt = f"Can you improve my code: '''{code}'''? (in a more pythonic way)"

print(get_completion(prompt))

The most Pythonic way to remove items with a specific value from a list is using a list comprehension:

```python
def some_function(input_list):
    return [item for item in input_list if item != 2]

list_01 = [1, 2, 4, 5, 6]
new_list = some_function(list_01)
print(new_list)  # Output: [1, 4, 5, 6]
```

This single line accomplishes the same task as your original `for` loop, in a more concise and readable manner.  It also avoids creating an unnecessary empty list beforehand.  Note that I've also corrected a typo, changing `input` to `input_list` in the function definition.



## Prompt: "What best practices should I use for XYZ?"

In [14]:
prompt = """I'm building my own user authentication with Python and FastAPI. 
Can you give me some 'best practices' to use here?"""

print(get_completion(prompt))

Building your own user authentication in Python with FastAPI requires careful attention to security best practices.  Rolling your own auth system is generally discouraged unless you have very specific needs not met by existing solutions (like Auth0, Firebase, etc.), but if you proceed, here's a breakdown of best practices:

**I. Password Handling:**

* **Hashing:**  **Never** store passwords in plain text.  Use a strong, one-way hashing algorithm like bcrypt or Argon2.  These are designed to be computationally expensive to crack, even with rainbow tables.  Python's `bcrypt` library is a good choice.  Avoid MD5 and SHA-1; they are considered insecure.

* **Salting:**  Always salt your passwords before hashing.  A salt is a random string added to the password before hashing.  This prevents attackers from pre-computing hashes for common passwords.  The salt should be unique for each password and stored alongside the hashed password.

* **Key Stretching:**  Use a sufficiently high iteratio

## Prompt: "What are the trade-offs?"

In [15]:
prompt = f"""
I am developing a new project utilizing a client-server architecture, implementing microservices. 
I am familiar with the challenges related to service discovery. 
Could you elucidate the trade-offs associated with the various approaches to service discovery?
"""

print(get_completion(prompt))

Let's explore the trade-offs of different service discovery approaches in a microservices architecture.  The key aspects to consider are:

**1. Client-side vs. Server-side Discovery:**

* **Client-side discovery:** Clients are responsible for finding the addresses of services.  This often uses a service registry (like Consul, etcd, ZooKeeper, or even a simple database).

    * **Pros:**
        * **Simplicity (potentially):**  Can be simpler to implement initially, especially with less complex deployments.
        * **Resilience (potentially):** Clients can have built-in retries and fallback mechanisms.
        * **Flexibility:** Clients can choose their load balancing strategies.
    * **Cons:**
        * **Increased client complexity:** Clients need to handle service discovery logic, increasing their codebase and testing burden.
        * **Tight coupling:** Clients are directly coupled to the service registry.  A change in the registry's format or location requires client updates.
 

## Prompt: "Using X, Y, Z, build me a ..."

In [16]:
prompt = f"""
Using codespell, black and isort create a pre-commit configuration file for python using pre-commit library
"""
print(get_completion(prompt))

```yaml
repos:
-   repo: https://github.com/codespell-project/codespell
    rev: v2.2.1
    hooks:
    -   id: codespell
        stages: [commit]
        args:
          - --quiet
          - --ignore-words-file=.codespell-ignore

-   repo: https://github.com/psf/black
    rev: 23.1.0
    hooks:
    -   id: black
        stages: [commit]
        args:
          - --line-length=79

-   repo: https://github.com/pycqa/isort
    rev: 5.12.0
    hooks:
    -   id: isort
        stages: [commit]
        args:
          - --profile=black

```

**Explanation:**

* **`repos`:** This section lists the repositories from which pre-commit will install hooks.
* **`repo`:** Specifies the Git repository URL for each tool (codespell, black, isort).  It's crucial to use a specific, stable version (like `rev: v2.2.1` or `rev: 23.1.0`) rather than `main` or `master` to avoid unexpected changes in the tool's behavior over time.
* **`rev`:**  Specifies the commit or tag to check out from the repository. Usi

## Prompt: "Transform this code into..."

In [17]:
code = """
#!/bin/bash

# Bash script to read a file line by line

filename="README.md"

# Check if the file exists
if [[ -f "$filename" ]]; then
    while IFS= read -r line
    do
        echo "$line"
    done < "$filename"
else
    echo "File not found!"
fi
"""
prompt = f"""
Can you transform this bash code: {{{code}}} to Python?
"""

print(get_completion(prompt))

Several ways exist to translate the bash script to Python. Here are two options, one using a `try-except` block for error handling and another demonstrating a more concise approach:


**Option 1:  Using `try-except` for error handling**

This option mirrors the bash script's structure more closely:


```python
filename = "README.md"

try:
    with open(filename, 'r') as file:
        for line in file:
            print(line, end='') # end='' prevents extra newline
except FileNotFoundError:
    print("File not found!")

```

This version explicitly handles the `FileNotFoundError` exception, making it robust to situations where the file doesn't exist.  The `end=''` in the `print()` function prevents adding an extra newline character, matching the bash script's behavior.


**Option 2: More concise approach**

This approach leverages Python's exception handling and file iteration capabilities for a more compact solution:


```python
filename = "README.md"

try:
    with open(filename) as f

## Prompt: "Write a bash script"

In [18]:
prompt = """
Write me a bash script that will find all files in my ~/Desktop directory that end in jpg, JPG, png, or PNG 
and copy them to the ~/Documents/tmp folder.
"""

print(get_completion(prompt))

```bash
#!/bin/bash

# Source and destination directories
source_dir="$HOME/Desktop"
dest_dir="$HOME/Documents/tmp"

# Create the destination directory if it doesn't exist
mkdir -p "$dest_dir"

# Find and copy the image files
find "$source_dir" -type f \( -iname "*.jpg" -o -iname "*.png" \) -exec cp {} "$dest_dir" \;

# Check if any files were copied
if [ $? -eq 0 ]; then
  echo "Image files copied successfully to $dest_dir"
else
  echo "Error copying image files."
fi

```

This script does the following:

1. **Sets variables:**  It defines variables for the source and destination directories using `$HOME` for portability.
2. **Creates destination directory:** It uses `mkdir -p` to create the `~/Documents/tmp` directory if it doesn't already exist.  The `-p` option creates parent directories as needed.
3. **Finds and copies files:**  `find` searches the source directory for files (`-type f`) that match the specified patterns.  `\( -iname "*.jpg" -o -iname "*.png" \)` uses `-iname` for 

## Prompt: "Build me a script in [language] that..."

In [19]:
prompt = """
Write a script in python that performs the following steps:

1. Goes to a YouTube channel
2. Finds the top 10 videos by view count
3. Saves the title, a URL to the thumbnail, and view count in a file called `youtube.txt`
"""

print(get_completion(prompt))

This script requires the `requests` and `beautifulsoup4` libraries.  Make sure you have them installed (`pip install requests beautifulsoup4`).  **This script scrapes data, and YouTube's terms of service may prohibit this. Use responsibly and at your own risk.**

```python
import requests
from bs4 import BeautifulSoup

def get_top_ten_youtube_videos(channel_url):
    """
    Scrapes the top 10 videos from a YouTube channel based on view count.

    Args:
        channel_url: The URL of the YouTube channel.

    Returns:
        A list of dictionaries, where each dictionary contains the title, thumbnail URL, and view count of a video.  Returns None if there's an error.
    """

    try:
        response = requests.get(channel_url)
        response.raise_for_status()  # Raise an exception for bad status codes

        soup = BeautifulSoup(response.content, 'html.parser')

        # YouTube's structure changes frequently.  This part is fragile and might break.
        #  Inspect the chann